In [1]:
#P2
import pennylane as qml
import numpy as np

In [2]:
def U_power_2k(unitary, k):
    """ Computes U at a power of 2k (U^2^k)
    
    Args: 
        unitary (array[complex]): A unitary matrix
    
    Returns: 
        array[complex]: the unitary raised to the power of 2^k
    """
    return np.linalg.matrix_power(unitary,np.power(2,k)) 
    pass
            

# Try out a higher power of U
U = qml.T.compute_matrix()
print(U)

U_power_2k(U, 2)

[[1.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]


array([[ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j]])

In [1]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def apply_controlled_powers_of_U(unitary):
    """A quantum function that applies the sequence of powers of U^2^k to
    the estimation wires.
    
    Args: 
        unitary (array [complex]): A unitary matrix
    """

    for i in range(len(estimation_wires)):
	    qml.ControlledQubitUnitary(U_power_2k(unitary,3 - (i+1)), control_wires=estimation_wires[i],wires=target_wires)
    pass

In [4]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """ Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Measurement outcome probabilities on the estimation wires.
    """
    prepare_eigenvector()
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
    apply_controlled_powers_of_U(unitary)
    qml.adjoint(qml.QFT)(wires=estimation_wires)

    return qml.probs(wires=estimation_wires)
    

U = qml.T.compute_matrix()
print(qpe(U))

[1.51544285e-32 1.00000000e+00 1.86828208e-31 9.26372303e-32
 9.07320326e-32 9.32150093e-32 1.85684709e-31 3.94430453e-31]


In [5]:
estimation_wires = [0, 1, 2]
target_wires = [3]

def estimate_phase(probs):
    """Estimate the value of a phase given measurement outcome probabilities
    of the QPE routine.
    
    Args: 
        probs (array[float]): Probabilities on the estimation wires.
    
    Returns:
        float: the estimated phase   
    """
    fact = 2**-3
    array1 = np.zeros(8)

    for i in range(8):
	    array1[i]= i * fact
    return float(sum(array1 * probs))

U = qml.T.compute_matrix()

probs = qpe(U)


estimated_phase = estimate_phase(probs)
print(estimated_phase)

0.1250000000000001


In [6]:
dev = qml.device("default.qubit", wires=4)

estimation_wires = [0, 1, 2]
target_wires = [3]

def prepare_eigenvector():
    qml.PauliX(wires=target_wires)

@qml.qnode(dev)
def qpe(unitary):
    """Estimate the phase for a given unitary.
    
    Args:
        unitary (array[complex]): A unitary matrix.
        
    Returns:
        array[float]: Probabilities on the estimation wires.
    """
    
    prepare_eigenvector()
    
    qml.QuantumPhaseEstimation(unitary,target_wires,estimation_wires)
    return qml.probs(estimation_wires)
    pass


U = qml.T.compute_matrix()
probs = qpe(U)
print(estimate_phase(probs))

0.1250000000000001
